In [ ]:
using Revise
using ContGridMod
using Ferrite
using FerriteViz
using GLMakie
using SparseArrays
using LinearAlgebra
using Flux
using Random
using Plots
Makie.inline!(true);

In [ ]:
NTRAIN = 48;
NTEST = 12;

In [ ]:
grid, scale_factor = get_grid("../data/borders/euro_border.json", 0.1, "panta.msh");
dm = load_discrete_model("../data/ml/test_1.h5", scale_factor);
model = get_params(grid, .05, dm, κ=0.02, bfactor=50000., σ=0.01, bmin=1);


In [ ]:
# load all discrete models
trainingDiscMod = ContGridMod.DiscModel[];
testDiscMod = ContGridMod.DiscModel[];
for i=1:NTRAIN
    push!(trainingDiscMod, load_discrete_model(
        "../data/ml/training_" * string(i) * ".h5", scale_factor));
end
for i=1:NTEST
    push!(testDiscMod, load_discrete_model(
        "../data/ml/test_" * string(i) * ".h5", scale_factor));
end

In [ ]:
# Make sure that all slack busses are the same
slack = trainingDiscMod[1].id_gen[trainingDiscMod[1].id_slack]
for i=1:NTRAIN
    if trainingDiscMod[i].id_gen[trainingDiscMod[i].id_slack] != slack
        println(i)
    end
end
for i=1:NTEST
    if testDiscMod[i].id_gen[testDiscMod[i].id_slack] != slack
        println(i)
    end
end

In [ ]:
# Set up training and test sets
trainingTheta = zeros(size(trainingDiscMod[1].th, 1), NTRAIN);
testTheta = zeros(size(testDiscMod[1].th, 1), NTEST);
trainingP = zeros(getnnodes(grid), NTRAIN);
testP = zeros(getnnodes(grid), NTEST);
for i=1:NTRAIN
    update_model!(model, :p, trainingDiscMod[i], .05, κ=0.02, bfactor=50000., σ=0.01, bmin=1)
    stable_sol!(model)
    trainingP[:, i] = model.f₀;
    trainingTheta[:, i] = trainingDiscMod[i].th;
end
for i=1:NTEST
    update_model!(model, :p, testDiscMod[i], .05, κ=0.02, bfactor=50000., σ=0.01, bmin=1)
    stable_sol!(model)
    testP[:, i] = model.f₀;
    testTheta[:, i] = testDiscMod[i].th;
end

In [ ]:
# Create matrices to obtain the mass matrix using just matrix multiplication and updating the susceptances in the quadrature points.
A = zeros(ndofs(model.dh₁), 2 * getnquadpoints(model.cellvalues) * size(model.grid.cells,1))
B = zeros(2 * getnquadpoints(model.cellvalues) * size(model.grid.cells,1), 2 * getnquadpoints(model.cellvalues) * size(model.grid.cells,1))
q_coords = zeros(2 * getnquadpoints(model.cellvalues) * size(model.grid.cells,1), 2)
n_basefuncs = getnbasefunctions(model.cellvalues)
for (i, cell) in enumerate(CellIterator(model.dh₁))
    Ferrite.reinit!(model.cellvalues, cell)    
    dofs = celldofs(cell)
    for q_point in 1:getnquadpoints(model.cellvalues)
        x = spatial_coordinate(model.cellvalues, q_point, getcoordinates(cell))
        dΩ = getdetJdV(model.cellvalues, q_point)
        ix = 2 * (i - 1) * getnquadpoints(model.cellvalues) + 2 * q_point - 1
        q_coords[ix, :] = q_coords[ix+1, :] = x
        for j in 1:n_basefuncs
            ∇φⱼ = shape_gradient(model.cellvalues, q_point, j)
            A[dofs[j], ix:ix+1] = ∇φⱼ * sqrt(dΩ)
    end
end
end
A[model.ch.prescribed_dofs, :] .= 0
A = sparse(A)
dim = zeros(ndofs(model.dh₁), ndofs(model.dh₁))
dim[model.ch.prescribed_dofs, model.ch.prescribed_dofs] .= 1
dim = sparse(dim);

In [ ]:
for (i, cell) in enumerate(CellIterator(model.dh₁))
    if i==2
        break
    end
    println(spatial_coordinate(model.cellvalues, 1, getcoordinates(cell))...)
end

In [ ]:
# The values to compare to the ground truth are obtained by interpolating between nodal values.
# This can be broken down to matrix multiplication of a projection matrix and the nodal values
func_interpolations = Ferrite.get_func_interpolations(model.dh₁, :u)
proj = zeros(size(dm.th, 1), ndofs(model.dh₁))
q_proj = zeros(size(q_coords, 1), 2 * ndofs(model.dh₁))
grid_coords = [node.x for node in grid.nodes] 

for i = 1:size(dm.th, 1)
    ph = PointEvalHandler(model.grid, [Ferrite.Vec(dm.coord[i, :]...)], warn=:false)
    if ph.cells[1] === nothing
        min_ix = argmin([norm(coord .- Ferrite.Vec(dm.coord[i, :]...)) for coord in grid_coords]) 
        ph = PointEvalHandler(grid, [grid_coords[min_ix]])
    end
    pv = Ferrite.PointScalarValuesInternal(ph.local_coords[1], func_interpolations[1])
    cell_dofs = Vector{Int}(undef, ndofs_per_cell(model.dh₁, ph.cells[1]))
    Ferrite.celldofs!(cell_dofs, model.dh₁, ph.cells[1])
    n_base_funcs = getnbasefunctions(pv)
    for j = 1:n_base_funcs
        proj[i, cell_dofs[j]] = shape_value(pv, 1, j)
    end
end
for (i, point) in enumerate(eachrow(q_coords))
    ph = PointEvalHandler(model.grid, [Ferrite.Vec(point...)])
    pv = Ferrite.PointScalarValuesInternal(ph.local_coords[1], func_interpolations[1])
    cell_dofs = Vector{Int}(undef, ndofs_per_cell(model.dh₁, ph.cells[1]))
    Ferrite.celldofs!(cell_dofs, model.dh₁, ph.cells[1])
    n_base_funcs = getnbasefunctions(pv)
    for j = 1:n_base_funcs
        if mod(i, 2) == 0
            q_proj[i, 2 * cell_dofs[j]] = shape_value(pv, 1, j)
        else
            q_proj[i, 2 * cell_dofs[j] - 1] = shape_value(pv, 1, j)
        end
    end
end
q_proj = sparse(q_proj);
proj = sparse(proj);

In [ ]:
# Set up the machine learning
N = getnnodes(model.grid)
opt = ADAM(0.1)
b = 100 * rand(2 * ndofs(model.dh₁)) .+ 0.1
param = Flux.params(b);
nEpochs = 5000;
nBatches = 3;
batchSize = Int64(NTRAIN / nBatches);
shuffledIx = randperm(NTRAIN);

In [ ]:
# Actual learning
err = zeros(nEpochs * nBatches)
for e=1:nEpochs
    for batch=1:nBatches
        local _err
        gs = Flux.gradient(param) do
            btemp = max.(b, 0.1)
            K = sparse(A * diagm(q_proj * btemp) * A' + dim)
            θ = proj * (K \ trainingP[:, shuffledIx[(batch - 1) * batchSize + 1:batch * batchSize]])
            _err = mean(abs2, θ .- trainingTheta[:, shuffledIx[(batch - 1) * batchSize + 1:batch * batchSize]])
            return _err
        end
        if(mod(e,50) == 0 && batch == 1)
            println([e _err])
        end
        err[(e - 1) * nBatches + batch] = _err
        Flux.update!(opt, param, gs)
    end
end
b = max.(b, 0.1);

In [ ]:
K = A * spdiagm(q_proj * b) * A' + dim
trainingThetaPred =  proj * (K \ trainingP)


In [ ]:
Plots.plot(err)

In [ ]:
update_model!(model, :bx, b[1:2:end])
update_model!(model, :by, b[2:2:end])

In [ ]:
# Project values from quadrautre points onto nodal values
# Note: a positive value in the quadrature points does NOT guarantee 
l2_proj = L2Projector(Ferrite.get_func_interpolations(model.dh₁, :u)[1], model.grid);
bx = b[1:2:end]
by = b[2:2:end]
bx = [Ferrite.Vec(bx[(i - 1) * 3 + 1], bx[(i - 1) * 3 + 2], bx[(i - 1) * 3 + 3]) for i in 1:size(bx, 1)÷3];
by = [Ferrite.Vec(by[(i - 1) * 3 + 1], by[(i - 1) * 3 + 2], by[(i - 1) * 3 + 3]) for i in 1:size(by, 1)÷3]
bx_nodal = project(l2_proj, bx, QuadratureRule{2,RefTetrahedron}(2), project_to_nodes=false)
by_nodal = project(l2_proj, by, QuadratureRule{2,RefTetrahedron}(2), project_to_nodes=false)
update_model!(model, :bx, bx_nodal);
update_model!(model, :by, by_nodal);

In [ ]:
nodal_plot(model, :bx_nodal)

In [ ]:
nodal_plot(model, :by_nodal)

In [ ]:
stable_sol!(model)
nodal_plot(model, :θ₀_nodal)

In [ ]:
# Get the predictions for all disc models


In [ ]:
ContGridMod.disc_plot(dm.coord[:,[2,1]], trainingDiscMod[NTRAIN].th)

In [ ]:
trainingThetaPred = zeros(size(trainingDiscMod[1].th, 1), NTRAIN);
testThetaPred = zeros(size(testDiscMod[1].th, 1), NTEST);
for i=1:NTRAIN
    update_model!(model, :p, trainingDiscMod[i], .05, κ=0.02, bfactor=50000., σ=0.01, bmin=1)
    stable_sol!(model)
    trainingThetaPred[:, i] = proj * model.θ₀_nodal;
end
for i=1:NTEST
    update_model!(model, :p, testDiscMod[i], .05, κ=0.02, bfactor=50000., σ=0.01, bmin=1)
    stable_sol!(model)
    testThetaPred[:, i] = proj * model.θ₀_nodal;
end

In [ ]:
using DelimitedFiles
writedlm("bxlearned.csv", b[1:2:end], ',')
writedlm("bylearned.csv", b[2:2:end], ',')

In [ ]:
trainingPlots = Plots.Plot[]

for i=1:NTRAIN
    min, max = extrema([trainingTheta[:,i] trainingThetaPred[:,i]])
    delta = max - min
    pad = 0.05 * delta
    min -= pad
    max += pad
push!(trainingPlots, Plots.scatter(trainingTheta[:,i], trainingThetaPred[:,i], xlims=(min, max), ylims=(min, max), label="Training Set " * string(i)))
end

Plots.plot(trainingPlots..., layout=(12,4), size=(1500, 3000))

In [ ]:
testPlots = Plots.Plot[]

for i=1:NTEST
    min, max = extrema([testTheta[:,i] testThetaPred[:,i]])
    delta = max - min
    pad = 0.05 * delta
    min -= pad
    max += pad
push!(testPlots, Plots.scatter(testTheta[:,i], testThetaPred[:,i], xlims=(min, max), ylims=(min, max), label="Test Set " * string(i)))
end

Plots.plot(testPlots..., layout=(4,3), size=(1125, 1000))

In [ ]:
add_local_disturbance!(model, [0.25, -0.1], -9., 0.05)
println("Synchronized freq: ", ContGridMod.integrate(model.dh₁, model.cellvalues, model.fault) / ContGridMod.integrate(model.dh₁, model.cellvalues, model.d))
sol = perform_dyn_sim(model, 50.);
save_simulation(model, sol, "fault")